In [1]:
# !pip install opencv-python moviepy numpy

In [12]:
import cv2
import numpy as np
from moviepy import *
import random 

def cartoonize_frame(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply median blur to reduce noise and smooth the image
    gray = cv2.medianBlur(gray, 5)
    
    # Detect edges using adaptive thresholding
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                  cv2.THRESH_BINARY, 9, 9)
    
    # Apply bilateral filter to smooth the image while keeping edges sharp
    color = cv2.bilateralFilter(frame, 9, 300, 300)
    
    # Combine the edges with the smoothed color image
    cartoon = cv2.bitwise_and(color, color, mask=edges)
    
    return cartoon



In [14]:
#useless
def pencil_sketch(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Invert the grayscale image
    inverted = cv2.bitwise_not(gray)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(inverted, (21, 21), 0)
    
    # Blend the grayscale image with the blurred inverted image
    sketch = cv2.divide(gray, 255 - blurred, scale=256)
    
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)
 

In [15]:
# helpful need modification
def classic_cartoon(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    
    # Smooth the image
    smoothed = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    
    # Combine edges and smoothed image
    cartoon = cv2.bitwise_and(smoothed, smoothed, mask=edges)
    
    return cartoon

In [18]:
def comic_book(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    
    # Posterize the image
    posterized = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    posterized[:, :, 2] = cv2.equalizeHist(posterized[:, :, 2])
    posterized = cv2.cvtColor(posterized, cv2.COLOR_HSV2BGR)
    
    # Combine edges and posterized image
    comic = cv2.bitwise_and(posterized, posterized, mask=edges)
    
    return comic

In [20]:
def watercolor_effect(image):
    # Apply median blur to smooth the image
    smoothed = cv2.medianBlur(image, 7)
    
    # Apply edge-preserving smoothing
    watercolor = cv2.stylization(smoothed, sigma_s=60, sigma_r=0.6)
    
    return watercolor

In [22]:
def oil_painting(image):
    # Apply oil painting effect
    oil_paint = cv2.xphoto.oilPainting(image, size=7, dynRatio=1)
    
    return oil_paint

In [24]:
#useless
def pop_art(image):
    # Reduce the number of colors using quantization
    data = np.float32(image).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    _, labels, centers = cv2.kmeans(data, 4, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()]
    quantized = quantized.reshape(image.shape)
    
    # Enhance edges (single-channel mask)
    edges = cv2.Canny(image, 100, 200)
    
    # Combine quantized image and edges
    pop_art = cv2.bitwise_and(quantized, quantized, mask=edges)
    
    return pop_art

In [33]:

def add_random_effect1(image,effect=None):
    # Get image dimensions
    height, width = image.shape[:2]
    total_pixels = height * width
    
    # Randomly select an effect
    if effect == None:
        effect = random.choice(['shape', 'text', 'rotation', 'border','color'])
        
    # print("Applying effect ",effect)
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        
    if effect == 'shape':
        # Randomly choose a shape: rectangle, circle, or line
        shape = random.choice(['rectangle', 'circle', 'line'])
        
        # Random color
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        
        if shape == 'rectangle':
            # Ensure the rectangle covers at least half of the image
            rect_width = random.randint(width // 2, width)
            rect_height = random.randint(height // 2, height)
            x1 = random.randint(0, width - rect_width)
            y1 = random.randint(0, height - rect_height)
            x2 = x1 + rect_width
            y2 = y1 + rect_height
            thickness = random.choice([-1, random.randint(1, 5)])  # -1 for filled rectangle
            cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        
        elif shape == 'circle':
            # Ensure the circle covers at least half of the image
            radius = random.randint(min(width, height) // 4, min(width, height) // 2)
            center_x = random.randint(radius, width - radius)
            center_y = random.randint(radius, height - radius)
            thickness = random.choice([-1, random.randint(1, 5)])  # -1 for filled circle
            cv2.circle(image, (center_x, center_y), radius, color, thickness)
        
        elif shape == 'line':
            # Ensure the line covers at least half of the image diagonally
            pt1 = (random.randint(0, width // 2), random.randint(0, height // 2))
            pt2 = (random.randint(width // 2, width), random.randint(height // 2, height))
            thickness = random.randint(1, 5)
            cv2.line(image, pt1, pt2, color, thickness)
    
    elif effect == 'text':
        # Ensure the text covers at least half of the image
        text = "Random Text"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = random.uniform(2.0, 5.0)  # Larger font scale to cover more area
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        thickness = random.randint(6, 10)
        
        # Calculate text size to ensure it covers enough area
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
        position = (random.randint(0, width - text_width%width), random.randint((text_height+1)%height, height))
        cv2.putText(image, text, position, font, font_scale, color, thickness)
    
    elif effect == 'rotation':
        # Rotate the image by a random angle
        angle = random.uniform(-180, 180)
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        image = cv2.warpAffine(image, rotation_matrix, (width, height))

    elif effect == 'color':        
          rect_width = random.randint(width // 2, width)
          rect_height = random.randint(height // 2, height)
          x1 = random.randint(0, width - rect_width)
          y1 = random.randint(0, height - rect_height)
          x2 = x1 + rect_width
          y2 = y1 + rect_height
          thickness = random.choice([-1, random.randint(25, 25)])  # -1 for filled rectangle
          cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        

          # w = width + 20
          # h = height + 20
          # cv2.rectangle(image, (70, 70), (width-70, height-70), (0, 0, 0), -1)
          # cv2.circle(image,(width // 2, height // 2), 200, (255, 0, 0), -1)

        
    elif effect == 'border':
        # Ensure the border covers at least half of the image
        border_thickness = random.randint(min(width, height) // 4, min(width, height) // 2)
        border_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        image = cv2.copyMakeBorder(image, border_thickness, border_thickness, border_thickness, border_thickness, cv2.BORDER_CONSTANT, value=border_color)
    
    return image


In [42]:
   
def video_to_cartoon(input_video_path, output_video_path):
    # Load the video
    clip = VideoFileClip(input_video_path)
    
    # Process each frame to cartoonize it
    cartoon_frames = [comic_book(frame) for frame in clip.iter_frames()]
    # add_random_effect1
    print(" ===========>  ",len(cartoon_frames))
    for f_idx in range(int(len(cartoon_frames))):
        if f_idx%60==0:
            # Create a copy of the image
            opacity = 0.2
            overlay = cartoon_frames[f_idx].copy()
            overlay = add_random_effect1(overlay,'color')
            result = cv2.addWeighted(overlay, opacity, cartoon_frames[f_idx], 1 - opacity, 0)

            cartoon_frames[f_idx] = result
            cv2.imwrite("Frame"+str(f_idx)+'.jpg', cartoon_frames[f_idx])
            

        
    # Create a new video from the cartoonized frames
    cartoon_clip = ImageSequenceClip(cartoon_frames, fps=clip.fps)
    
    # Write the output video
    cartoon_clip.write_videofile(output_video_path, codec='libx264')
    

In [45]:

# Example usage
input_video = '9sec.mp4'
output_video = 'cartoon_9sec.mp4'
video_to_cartoon(input_video, output_video)
output_clip = VideoFileClip(output_video)  
# output_clip.ipython_display(width = 360) 

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '512', 'compatible_brands': 'mp42isom', 'creation_time': '2025-03-18T11:58:20.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [960, 540], 'bitrate': 1313, 'fps': 60.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'creation_time': '2025-03-18T11:58:20.000000Z', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'h264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 192, 'metadata': {'Metadata': '', 'creation_time': '2025-03-18T11:58:20.000000Z', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 9.75, 'bitrate': 1531, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'vi

MoviePy - Done !
MoviePy - video ready cartoon_9sec.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 540], 'bitrate': 3301, 'fps': 60.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 9.75, 'bitrate': 3307, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [960, 540], 'video_bitrate': 3301, 'video_fps': 60.0, 'video_duration': 9.75, 'video_n_frames': 585}
C:\Users\DELL\anaconda3\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i cartoon_9sec.mp4 -logle